## HTML Table Customization
#### Goal 
- Extend the functionality of `df.to_html()` to allow for html attributes to be added to any element in the table html.

#### Current Status
- The ability to assign specific Bootstrap classes for text alignment has been added. Currently these classes are hard coded in.
- Several different types of output table html are supported:
    - tables with simple (1 level) indexes
    - tables with multi-indexes (hierarchical indexes)
    - tables with index names
    - tables with the row index removed
- For text alignment, only the following data types are accounted for:
    - string
    - boolean
    - numeric

#### Future Work
- Add additional functions, as needed, to achieve other customizations. As an example, functions to add different background and foreground colors to highlight rows/columns/data in the table.
- Along the lines of text alignment, account for other data types and their ideal horizontal alignment.
- Create tests using the 3rd-party library hypothesis to generate test DataFrames in a more automated way, covering more scenarios specifically concerning the structure of indexes. Then adjust the code to handle these scenarios.

In [1]:
from pathlib import Path
from typing import Any, TypeAlias

import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas.api.types import (
    is_bool_dtype,
    is_numeric_dtype,
    is_object_dtype,
    is_string_dtype,
)
from selectolax.parser import HTMLParser

### Test DataFrames

In [2]:
# Creating a sample dataset with multi-level columns.
data = {
    ("Score", "Midterm"): [25, 30, 35, 40, 45, 50, 55, 34, 60],
    ("Score", "Final"): [28, 33, 38, 43, 48, 53, 58, 34, 63],
    ("Sport", "Baseball"): [28, 33, 38, 43, 48, 53, 58, 34, 63],
    ("Attendance", "Days Present"): [20, 22, 18, 19, 24, 2, 21, 23, 20],
    ("Attendance", "Days Absent"): [0, 2, 2, 1, 0, 1, 2, 1, 2],
    ("Other", "Misc"): [28, 33, 38, 43, 48, 53, 58, 34, 63],
}

# Creating a MultiIndex for the rows (as before).
multi_index_rows = pd.MultiIndex.from_tuples(
    [
        ("Student A", "Math"),
        ("Student A", "Science"),
        ("Student B", "Math"),
        ("Student B", "Science"),
        ("Student C", "Math"),
        ("Student D", "Math"),
        ("Student D", "Science"),
        ("Student D", "Math"),
        ("Student E", "Science"),
    ],
    names=["Student", "Subject"],
)

# Creating a MultiIndex for the columns.
multi_index_cols = pd.MultiIndex.from_tuples(
    [
        ("Score", "Midterm"),
        ("Score", "Final"),
        ("Sport", "Baseball"),
        ("Attendance", "Days Present"),
        ("Attendance", "Days Absent"),
        ("Other", "Misc"),
    ],
    names=["Category", "Type"],
)

# Creating the DataFrame.
df = pd.DataFrame(data, index=multi_index_rows, columns=multi_index_cols)
df[("Score", "Midterm")] = 3.0
df[("Attendance", "Days Present")] = True
df[("Score", "Final")] = "yes"
df[("Other", "Misc")] = pd.Series(data=range(len(df)), index=df.index)
df[("Other", "Misc")] = "a string"
df

Category            Score          Sport   Attendance                 Other
Type              Midterm Final Baseball Days Present Days Absent      Misc
Student   Subject                                                          
Student A Math        3.0   yes       28         True           0  a string
          Science     3.0   yes       33         True           2  a string
Student B Math        3.0   yes       38         True           2  a string
          Science     3.0   yes       43         True           1  a string
Student C Math        3.0   yes       48         True           0  a string
Student D Math        3.0   yes       53         True           1  a string
          Science     3.0   yes       58         True           2  a string
          Math        3.0   yes       34         True           1  a string
Student E Science     3.0   yes       63         True           2  a string

In [3]:
# Creating a MultiIndex with mixed data types
multi_index = pd.MultiIndex.from_tuples(
    [
        ("A", 1),
        ("A", 2),
        ("B", 1),
        ("B", 2),
    ],
    names=["String Level", "Integer Level"],
)

# Creating a DataFrame using the MultiIndex
df2 = pd.DataFrame({"Data": [100, 200, 300, 400]}, index=multi_index)
df2

Data
String Level Integer Level      
A            1               100
             2               200
B            1               300
             2               400

In [4]:
# Define the multi-index for columns
multi_index = pd.MultiIndex.from_tuples(
    [("Group A", "Data 1"), ("Group A", "Data 2"), ("Group B", "Data 1")],
    names=[1212, 1333],  # Only the second level has a name
)

# Sample data
data = {
    ("Group A", "Data 1"): [1, 2, 3],
    ("Group A", "Data 2"): [4, 5, 6],
    ("Group B", "Data 1"): [7, 8, 9],
}

# Create the DataFrame
df3 = pd.DataFrame(data, index=["Row 1", "Row 2", "Row 3"], columns=multi_index)

df3

1212  Group A        Group B
1333   Data 1 Data 2  Data 1
Row 1       1      4       7
Row 2       2      5       8
Row 3       3      6       9

In [5]:
df4 = pd.DataFrame(range(1, 8))
df4

,0
0,1
1,2
2,3
3,4
4,5
5,6
6,7


In [6]:
# Creating a sample dataset with multi-level columns.
data = {
    ("Score", "Midterm"): [25, 30, 35, 40, 45, 50, 55, 34, 60],
    ("Score", "Final"): [28, 33, 38, 43, 48, 53, 58, 34, 63],
    ("Sport", "Baseball"): [28, 33, 38, 43, 48, 53, 58, 34, 63],
    ("Attendance", "Days Present"): [20, 22, 18, 19, 24, 2, 21, 23, 20],
    ("Attendance", "Days Absent"): [0, 2, 2, 1, 0, 1, 2, 1, 2],
    ("Other", "Misc"): [28, 33, 38, 43, 48, 53, 58, 34, 63],
}

# Creating a MultiIndex for the rows (as before).
multi_index_rows = pd.MultiIndex.from_tuples(
    [
        ("Student A", "Math"),
        ("Student A", "Science"),
        ("Student B", "Math"),
        ("Student B", "Science"),
        ("Student C", "Math"),
        ("Student D", "Math"),
        ("Student D", "Science"),
        ("Student D", "Math"),
        ("Student E", "Science"),
    ],
)

# Creating a MultiIndex for the columns.
multi_index_cols = pd.MultiIndex.from_tuples(
    [
        ("Score", "Midterm"),
        ("Score", "Final"),
        ("Sport", "Baseball"),
        ("Attendance", "Days Present"),
        ("Attendance", "Days Absent"),
        ("Other", "Misc"),
    ],
)

# Creating the DataFrame.
df5 = pd.DataFrame(data, index=multi_index_rows, columns=multi_index_cols)
df5[("Score", "Midterm")] = 3.0
df5[("Attendance", "Days Present")] = True
df5[("Score", "Final")] = "yes"
df5[("Other", "Misc")] = pd.Series(data=range(len(df)), index=df.index)
df5[("Other", "Misc")] = "a string"
df5

Score          Sport   Attendance                 Other
                  Midterm Final Baseball Days Present Days Absent      Misc
Student A Math        3.0   yes       28         True           0  a string
          Science     3.0   yes       33         True           2  a string
Student B Math        3.0   yes       38         True           2  a string
          Science     3.0   yes       43         True           1  a string
Student C Math        3.0   yes       48         True           0  a string
Student D Math        3.0   yes       53         True           1  a string
          Science     3.0   yes       58         True           2  a string
          Math        3.0   yes       34         True           1  a string
Student E Science     3.0   yes       63         True           2  a string

In [7]:
# Creating a sample dataset with multi-level columns.
data = {
    ("Score", "Midterm"): [25, 30, 35, 40, 45, 50, 55, 34, 60],
    ("Score", "Final"): [28, 33, 38, 43, 48, 53, 58, 34, 63],
    ("Sport", "Baseball"): [28, 33, 38, 43, 48, 53, 58, 34, 63],
    ("Attendance", "Days Present"): [20, 22, 18, 19, 24, 2, 21, 23, 20],
    ("Attendance", "Days Absent"): [0, 2, 2, 1, 0, 1, 2, 1, 2],
    ("Other", "Misc"): [28, 33, 38, 43, 48, 53, 58, 34, 63],
}

# Creating a MultiIndex for the rows (as before).
multi_index_rows = pd.MultiIndex.from_tuples(
    [
        ("Student A", "Math"),
        ("Student A", "Science"),
        ("Student B", "Math"),
        ("Student B", "Science"),
        ("Student C", "Math"),
        ("Student D", "Math"),
        ("Student D", "Science"),
        ("Student D", "Math"),
        ("Student E", "Science"),
    ],
)

# Creating a MultiIndex for the columns.
multi_index_cols = pd.MultiIndex.from_tuples(
    [
        ("Score", "Midterm"),
        ("Score", "Final"),
        ("Sport", "Baseball"),
        ("Attendance", "Days Present"),
        ("Attendance", "Days Absent"),
        ("Other", "Misc"),
    ],
    names=[None, "Hi"],
)

# Creating the DataFrame.
df6 = pd.DataFrame(data, index=multi_index_rows, columns=multi_index_cols)
df6[("Score", "Midterm")] = 3.0
df6[("Attendance", "Days Present")] = True
df6[("Score", "Final")] = "yes"
df6[("Other", "Misc")] = pd.Series(data=range(len(df)), index=df.index)
df6[("Other", "Misc")] = "a string"
df6

Score          Sport   Attendance                 Other
Hi                Midterm Final Baseball Days Present Days Absent      Misc
Student A Math        3.0   yes       28         True           0  a string
          Science     3.0   yes       33         True           2  a string
Student B Math        3.0   yes       38         True           2  a string
          Science     3.0   yes       43         True           1  a string
Student C Math        3.0   yes       48         True           0  a string
Student D Math        3.0   yes       53         True           1  a string
          Science     3.0   yes       58         True           2  a string
          Math        3.0   yes       34         True           1  a string
Student E Science     3.0   yes       63         True           2  a string

In [8]:
DATA_DIR = Path.cwd().parents[1] / "src" / "data"

df7 = pd.read_csv(DATA_DIR / "feature_importance.csv", index_col=0)
df7 = df7.iloc[:5]
# df["Bool"] = True
# df.iloc[2, 1] = False
df7

,Relative Importance
STL,1.00
BLK,0.93
OFFRTG,0.55
DREB,0.48
3P%,0.43


---

### Functions for Horizontal Alignment of Table Columns

In [9]:
Dtype: TypeAlias = np.dtype | pd.api.extensions.ExtensionDtype | type
DtypeIndices: TypeAlias = dict[str, list[int]]


def get_dtype_category(dtype: Dtype) -> str:
    """Get the data type category for `dtype`.

    Parameters
    ----------
    dtype : Dtype
        Object whose data type category is to be determined.

    Returns
    -------
    str
        Data type category.
    """
    # Columns inferred as "object" type will be treated as strings.
    if is_object_dtype(dtype) or is_string_dtype(dtype):
        return "string"
    elif is_bool_dtype(dtype):
        return "boolean"
    elif is_numeric_dtype(dtype):
        return "numeric"
    else:
        return "other"


def get_dtype_indices(dtypes: list[Dtype]) -> DtypeIndices:
    """Build dict of type category and indices where that type occurs in a DataFrame.

    Parameters
    ----------
    dtypes : list[Dtype]
        List containing data types of an index (row or column).

    Returns
    -------
    DtypeIndices
        Indices for each data type category with the categories as keys.
    """
    dtype_indices = {
        "string": [],
        "boolean": [],
        "numeric": [],
    }

    for index, dtype in enumerate(dtypes):
        dtype_category = get_dtype_category(dtype=dtype)
        if dtype_category in dtype_indices:
            dtype_indices[dtype_category].append(index)

    return dtype_indices


def combine_indices(
    index_indices: list[int], column_indices: list[int], index_levels: int
) -> list[int]:
    """Combine data type indices for the index and columns of a DataFrame into one list.

    The row index of an html table is being treated as another column or set of columns
    (if hierarchical) in addition to the normal columns. Therefore, data type indices
    for the row index (`index_indices`) need to be combined with the ones from the
    normal columns (`column_indices`). `index_levels` is used to shift the
    `column_indices` values to account for this new indexing (row index plus normal
    columns) prior to doing the combination.

    Note: Every index value (range(len(`index_indices`) + len(`column_indices`))) does
    not need to be present in the arguments or in the return value of this function. A
    given index value is only present if the corresponding "column" is of the given data
    type category currently being processed (see the calling function for the current
    data type category being processed).

    Parameters
    ----------
    index_indices : list[int]
        Indices of the row index.
    column_indices : list[int]
        Indices of the columns.
    index_levels : int
        Number of levels of the row index.

    Returns
    -------
    list[int]
        Combined indices.
    """
    combined_indices = index_indices.copy()
    # The list comprehension shifts the values of `column_indices` so that there is no
    # overlap in the values coming from `index_indices` with the values from
    # `column_indices` in the combination.
    combined_indices.extend([index + index_levels for index in column_indices])

    return combined_indices


def get_full_table_dtype_indices(df: DataFrame, index: bool) -> DtypeIndices:
    """Retrieve combined data type indices for the index and columns of `df`.

    Parameters
    ----------
    df : DataFrame
        DataFrame an html table will be created from.
    index : bool
        Boolean indicating whether or not to include the row index indices in the return
        value.

    Returns
    -------
    DtypeIndices
        Data categories and their indices within `df` (index and column indices
        combined).
    """
    index_dtypes: list[Dtype]

    if index is False:
        # If DataFrame has column names.
        if any(name is not None for name in df.columns.names):
            index_dtypes = [pd.Series(df.columns.names).dtype]
            # Column names take up an extra column to the left of the data columns,
            # therefore `index_levels` is set to 1 to account for this.
            index_levels = 1
        # If DataFrame does not have column names.
        else:
            index_dtypes = []
            index_levels = 0
    else:
        # Row index dtypes are accessed differently based on whether it is a MultiIndex
        # or not.
        if isinstance(df.index, pd.MultiIndex):
            index_dtypes = df.index.dtypes.to_list()
        else:
            index_dtypes = [df.index.dtype]

        index_levels = df.index.nlevels

    index_dtype_indices = get_dtype_indices(dtypes=index_dtypes)
    column_dtype_indices = get_dtype_indices(dtypes=df.dtypes)

    # The data structures are zipped in order to pass the data corresponding to the same
    # data type category to `combine_indices()`.
    zip_indices = zip(
        index_dtype_indices.keys(),
        index_dtype_indices.values(),
        column_dtype_indices.values(),
    )

    indices = {}

    for category, index_indices, column_indices in zip_indices:
        indices[category] = combine_indices(
            index_indices=index_indices,
            column_indices=column_indices,
            index_levels=index_levels,
        )

    return indices


def align_table_columns_horizontally(
    table_html: str, dtype_indices: DtypeIndices
) -> str:
    """Align columns horizontally with Bootstrap classes based on data type of column.

    Parameters
    ----------
    table_html : str
        Html string representing the table.
    dtype_indices : DtypeIndices
        Data type categories and the indices where they occur in the table.

    Returns
    -------
    str
        Updated table html string.
    """
    parser = HTMLParser(table_html)
    # Find all table rows including the header row(s).
    table_rows = parser.css("tr")

    rowspan_shift_remaining = 0

    for table_row in table_rows:
        # The amount of index shift that a row index node with a rowspan attribute
        # creates is 1 for each row (it is constant per row). The total number of index
        # shifts (i.e. total number of rows affected) is the value of the rowspan
        # attribute minus 1. The shift does not affect the row in which the rowspan
        # attribute is encountered, it affects only subsequent rows. As each row shift
        # is completed, `rowspan_shift_remaining` is decremented until 0 is reached.
        if rowspan_shift_remaining > 0:
            rowspan_shift_remaining -= 1
            rowspan_shift = 1
        else:
            rowspan_shift = 0
        # `colspan_shift` is reset for each row since column index nodes with a colspan
        # attribute only affect the row in which they show up.
        colspan_shift = 0

        for col, child_node in enumerate(table_row.iter()):
            if "colspan" in child_node.attributes:
                # Index shift of a colspan node is the colspan value minus 1. The +=
                # operator is used because if multiple colspan nodes are encountered
                # while still on the same row, index shift will stack for all nodes
                # remaining in that row.
                colspan_shift += int(child_node.attrs["colspan"]) - 1
                # Column index nodes with a colspan attribute are centered horizontally.
                child_node.attrs["class"] = "text-center"
                # Remove deprecated html attribute "halign".
                try:
                    del child_node.attrs["halign"]
                except KeyError:
                    print("The attribute 'halign' was not found within this node.")
            elif "rowspan" in child_node.attributes:
                # Index shift of rowspan node is the rowspan value minus 1.
                rowspan_shift_remaining = int(child_node.attrs["rowspan"]) - 1
                # Row index nodes with a colspan attribute are centered vertically.
                child_node.attrs["class"] = "align-middle"
                # Remove deprecated html attribute "valign".
                try:
                    del child_node.attrs["valign"]
                except KeyError:
                    print("The attribute 'valign' was not found within this node.")
            else:
                # Adjust the index by any shifts due to rowspan/colspan nodes.
                index = col + rowspan_shift + colspan_shift

                if index in dtype_indices["string"]:
                    # Left align strings.
                    child_node.attrs["class"] = "text-start"
                elif index in dtype_indices["boolean"]:
                    # Center align booleans.
                    child_node.attrs["class"] = "text-center"
                elif index in dtype_indices["numeric"]:
                    # Right align numerics.
                    child_node.attrs["class"] = "text-end"

    return parser.css_first("table").html


def convert_frame_to_html(df: DataFrame, **to_html_kwargs: Any) -> str:
    """Convert DataFrame to html table, include bootstrap classes for text alignment.

    Parameters
    ----------
    df : DataFrame
        DataFrame to be converted to an html table.
    **to_html_kwargs : Any
        Keyword arguments passed directly to `df.to_html()`.

    Returns
    -------
    str
        Html table string.
    """
    # Starting html table string.
    table_html = df.to_html(**to_html_kwargs)

    index = to_html_kwargs.get("index", True)
    if not isinstance(index, bool):
        raise TypeError("`index` must be of type `bool`.")

    dtype_indices = get_full_table_dtype_indices(df, index=index)

    updated_table_html = align_table_columns_horizontally(
        table_html=table_html, dtype_indices=dtype_indices
    )

    return updated_table_html

### Results/Testing

In [10]:
convert_frame_to_html(df=df, border=0, classes=["table", "table-dark", "table-striped"])

'<table class="dataframe table table-dark table-striped">\n  <thead>\n    <tr>\n      <th class="text-start"></th>\n      <th class="text-start">Category</th>\n      <th colspan="2" class="text-center">Score</th>\n      <th class="text-end">Sport</th>\n      <th colspan="2" class="text-center">Attendance</th>\n      <th class="text-start">Other</th>\n    </tr>\n    <tr>\n      <th class="text-start"></th>\n      <th class="text-start">Type</th>\n      <th class="text-end">Midterm</th>\n      <th class="text-start">Final</th>\n      <th class="text-end">Baseball</th>\n      <th class="text-center">Days Present</th>\n      <th class="text-end">Days Absent</th>\n      <th class="text-start">Misc</th>\n    </tr>\n    <tr>\n      <th class="text-start">Student</th>\n      <th class="text-start">Subject</th>\n      <th class="text-end"></th>\n      <th class="text-start"></th>\n      <th class="text-end"></th>\n      <th class="text-center"></th>\n      <th class="text-end"></th>\n      <th 

In [11]:
df

Category            Score          Sport   Attendance                 Other
Type              Midterm Final Baseball Days Present Days Absent      Misc
Student   Subject                                                          
Student A Math        3.0   yes       28         True           0  a string
          Science     3.0   yes       33         True           2  a string
Student B Math        3.0   yes       38         True           2  a string
          Science     3.0   yes       43         True           1  a string
Student C Math        3.0   yes       48         True           0  a string
Student D Math        3.0   yes       53         True           1  a string
          Science     3.0   yes       58         True           2  a string
          Math        3.0   yes       34         True           1  a string
Student E Science     3.0   yes       63         True           2  a string

In [12]:
get_full_table_dtype_indices(df, index=True)

{'string': [0, 1, 3, 7], 'boolean': [5], 'numeric': [2, 4, 6]}

In [13]:
get_full_table_dtype_indices(df, index=False)

{'string': [0, 2, 6], 'boolean': [4], 'numeric': [1, 3, 5]}

In [14]:
df2

Data
String Level Integer Level      
A            1               100
             2               200
B            1               300
             2               400

In [15]:
get_full_table_dtype_indices(df2, index=True)

{'string': [0], 'boolean': [], 'numeric': [1, 2]}

In [16]:
get_full_table_dtype_indices(df2, index=False)

{'string': [], 'boolean': [], 'numeric': [0]}

In [17]:
df3

1212  Group A        Group B
1333   Data 1 Data 2  Data 1
Row 1       1      4       7
Row 2       2      5       8
Row 3       3      6       9

In [18]:
get_full_table_dtype_indices(df3, index=True)

{'string': [0], 'boolean': [], 'numeric': [1, 2, 3]}

In [19]:
get_full_table_dtype_indices(df3, index=False)

{'string': [], 'boolean': [], 'numeric': [0, 1, 2, 3]}

In [20]:
df4

,0
0,1
1,2
2,3
3,4
4,5
5,6
6,7


In [21]:
get_full_table_dtype_indices(df4, index=True)

{'string': [], 'boolean': [], 'numeric': [0, 1]}

In [22]:
get_full_table_dtype_indices(df4, index=False)

{'string': [], 'boolean': [], 'numeric': [0]}

In [23]:
df5

Score          Sport   Attendance                 Other
                  Midterm Final Baseball Days Present Days Absent      Misc
Student A Math        3.0   yes       28         True           0  a string
          Science     3.0   yes       33         True           2  a string
Student B Math        3.0   yes       38         True           2  a string
          Science     3.0   yes       43         True           1  a string
Student C Math        3.0   yes       48         True           0  a string
Student D Math        3.0   yes       53         True           1  a string
          Science     3.0   yes       58         True           2  a string
          Math        3.0   yes       34         True           1  a string
Student E Science     3.0   yes       63         True           2  a string

In [24]:
get_full_table_dtype_indices(df5, index=True)

{'string': [0, 1, 3, 7], 'boolean': [5], 'numeric': [2, 4, 6]}

In [25]:
get_full_table_dtype_indices(df5, index=False)

{'string': [1, 5], 'boolean': [3], 'numeric': [0, 2, 4]}

In [26]:
df6

Score          Sport   Attendance                 Other
Hi                Midterm Final Baseball Days Present Days Absent      Misc
Student A Math        3.0   yes       28         True           0  a string
          Science     3.0   yes       33         True           2  a string
Student B Math        3.0   yes       38         True           2  a string
          Science     3.0   yes       43         True           1  a string
Student C Math        3.0   yes       48         True           0  a string
Student D Math        3.0   yes       53         True           1  a string
          Science     3.0   yes       58         True           2  a string
          Math        3.0   yes       34         True           1  a string
Student E Science     3.0   yes       63         True           2  a string

In [27]:
get_full_table_dtype_indices(df6, index=True)

{'string': [0, 1, 3, 7], 'boolean': [5], 'numeric': [2, 4, 6]}

In [28]:
get_full_table_dtype_indices(df6, index=False)

{'string': [0, 2, 6], 'boolean': [4], 'numeric': [1, 3, 5]}

In [29]:
df7

,Relative Importance
STL,1.00
BLK,0.93
OFFRTG,0.55
DREB,0.48
3P%,0.43


In [30]:
get_full_table_dtype_indices(df7, index=True)

{'string': [0], 'boolean': [], 'numeric': [1]}

In [31]:
get_full_table_dtype_indices(df7, index=False)

{'string': [], 'boolean': [], 'numeric': [0]}